In [ ]:
from google.colab import files
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# 파일 업로드
uploaded = files.upload()

image_path = list(uploaded.keys())[0]
input_format = os.path.splitext(image_path)[1][1:].upper()

if input_format == 'JPG':
    input_format = 'JPEG'

original_image = Image.open(image_path).convert('RGB')

# Gaussian 노이즈 추가 함수
def add_gaussian_noise(image, intensity=0.1):
    np_image = np.array(image, dtype=np.float32)
    mean = 0
    stddev = intensity * 255
    noise = np.random.normal(mean, stddev, np_image.shape)
    noisy_image = np.clip(np_image + noise, 0, 255)
    return Image.fromarray(noisy_image.astype(np.uint8))

# Salt-and-Pepper 노이즈 추가 함수
def add_salt_and_pepper_noise(image, intensity=0.05):
    np_image = np.array(image, dtype=np.uint8)
    prob = intensity
    noisy_image = np_image.copy()

    num_salt = np.ceil(prob * np_image.size * 0.5)
    num_pepper = np.ceil(prob * np_image.size * 0.5)

    coords_salt = [np.random.randint(0, i, int(num_salt)) for i in np_image.shape[:2]]
    noisy_image[coords_salt[0], coords_salt[1], :] = 255

    coords_pepper = [np.random.randint(0, i, int(num_pepper)) for i in np_image.shape[:2]]
    noisy_image[coords_pepper[0], coords_pepper[1], :] = 0

    return Image.fromarray(noisy_image)

# 슬라이더와 라벨 UI 구성
gaussian_slider = widgets.FloatSlider(
    value=0.1, min=0.05, max=0.3, step=0.01, description='Gaussian:', continuous_update=False
)
sap_slider = widgets.FloatSlider(
    value=0.01, min=0.01, max=0.1, step=0.01, description='Salt&Pepper:', continuous_update=False
)
intensity_label = widgets.Label(value="노이즈 강도를 설정하세요 (추천 범위 Gaussian: 0.05~0.3, Salt-and-Pepper: 0.01~0.1)")

apply_noise_button = widgets.Button(description="Apply Noise", button_style='info')

save_option_widget = widgets.RadioButtons(
    options=['Y', 'N'], description='저장 여부:', value=None, disabled=True
)

def save_image():
    global image_choice_widget  # 전역 변수로 선언
    image_choice_widget = widgets.RadioButtons(
        options=['Gaussian Noise', 'Salt-and-Pepper Noise', 'Combined Noise'],
        description='노이즈 방식:',
        value=None
    )

    save_name_widget = widgets.Text(
        value='', placeholder='파일 이름 입력', description='파일 이름:'
    )
    save_button = widgets.Button(description="저장", button_style='success')

    save_controls = widgets.HBox([save_name_widget, save_button])

    # 저장 버튼 클릭 이벤트
    def on_save_button_click(b):
        save_name = save_name_widget.value.strip()
        if not save_name:
            print("파일 이름을 입력해야 합니다.")
            return

        if image_choice_widget.value == 'Gaussian Noise':
            selected_image = gaussian_noisy_image
        elif image_choice_widget.value == 'Salt-and-Pepper Noise':
            selected_image = salt_and_pepper_noisy_image
        elif image_choice_widget.value == 'Combined Noise':
            selected_image = combined_noisy_image
        else:
            print("노이즈 방식을 선택해야 합니다.")
            return

        file_path = f"{save_name}.{input_format.lower()}"

        selected_image.save(file_path, format=input_format)
        print(f"{file_path} 파일이 저장되었습니다. \n")
        files.download(file_path)

        save_option_widget.disabled = False
        save_option_widget.value = None
        display(save_option_widget)

    save_button.on_click(on_save_button_click)

    def on_image_choice_change(change):
        if change['new']:
            display(save_controls)

    image_choice_widget.observe(on_image_choice_change, names='value')

    display(image_choice_widget)

# 저장 여부 변경 이벤트 처리
def on_save_option_change(change):
    if change['new'] == 'Y':
        save_image()
    elif change['new'] == 'N':
        print("저장을 종료합니다.")
        save_option_widget.disabled = True
        disable_noise_choice()
        reset_state(preserve_selection=True)

save_option_widget.observe(on_save_option_change, names='value')

# 노이즈 적용 버튼 이벤트
def apply_noise(b):
    gaussian_intensity = gaussian_slider.value
    sap_intensity = sap_slider.value

    # 노이즈 이미지 생성
    global gaussian_noisy_image, salt_and_pepper_noisy_image, combined_noisy_image
    gaussian_noisy_image = add_gaussian_noise(original_image, intensity=gaussian_intensity)
    salt_and_pepper_noisy_image = add_salt_and_pepper_noise(original_image, intensity=sap_intensity)
    combined_noisy_image = add_salt_and_pepper_noise(
        gaussian_noisy_image, intensity=sap_intensity
    )

    # 노이즈 결과 표시
    plt.figure(figsize=(18, 6))
    plt.subplot(1, 4, 1)
    plt.imshow(original_image)
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 4, 2)
    plt.imshow(gaussian_noisy_image)
    plt.title(f"Gaussian Noise ({gaussian_intensity})")
    plt.axis("off")

    plt.subplot(1, 4, 3)
    plt.imshow(salt_and_pepper_noisy_image)
    plt.title(f"Salt-and-Pepper Noise ({sap_intensity})")
    plt.axis("off")

    plt.subplot(1, 4, 4)
    plt.imshow(combined_noisy_image)
    plt.title(f"Combined Noise (G:{gaussian_intensity}, SP:{sap_intensity})")
    plt.axis("off")
    plt.show()

    save_option_widget.disabled = False
    display(save_option_widget)

def disable_noise_choice():
    global image_choice_widget
    if 'image_choice_widget' in globals() and image_choice_widget is not None:
        image_choice_widget.disabled = True

def reset_state(preserve_selection=False):
    global image_choice_widget
    if 'image_choice_widget' in globals() and image_choice_widget is not None:
        if not preserve_selection:
            image_choice_widget.value = None

apply_noise_button.on_click(apply_noise)

display(intensity_label, gaussian_slider, sap_slider, apply_noise_button)

In [ ]:
# 임시 작업 공간(/content) 파일 삭제

import os

# /content/ 디렉토리 내의 모든 파일을 삭제
directory_path = '/content/'

files = os.listdir(directory_path)

for file in files:
    file_path = os.path.join(directory_path, file)
    if os.path.isfile(file_path):
        os.remove(file_path)
        print(f"{file_path} 파일이 삭제되었습니다.")